In [1]:
from google.colab import files
files.upload()


Saving ReviewsFileName.xlsx to ReviewsFileName.xlsx


{'ReviewsFileName.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [3]:
import pandas as pd
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
df  = pd.read_excel("ReviewsFileName.xlsx")
df.head()

,Review,Sentiment
0,the rock is destined to be the 21st century's ...,0
1,"the gorgeously elaborate continuation of "" the...",0
2,effective but too-tepid biopic\n,0
3,if you sometimes like to go to the movies to h...,0
4,"emerges as something rare , an issue movie tha...",0


In [8]:
cachedStopWords = stopwords.words("english")
stemmer = PorterStemmer()

In [9]:
def pre_process_data(data):

    list_Sentiment = []
    list_Review = []
    len_data = len(df)
    i=0
    
    for row in df.iterrows():
        i+=1
    
        ##### Remove and clean comments
        posts = row[1].Review
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        temp = " ".join([stemmer.stem(w) for w in temp.split(' ') if w not in cachedStopWords])
        list_Sentiment.append(row[1].Sentiment)
        list_Review.append(temp)

    #del data
    list_Review = np.array(list_Review)
    list_Sentiment = np.array(list_Sentiment)
    return list_Review, list_Sentiment

list_Review, list_Sentiment = pre_process_data(df)

In [10]:
cntizer = CountVectorizer()
X_cnt = cntizer.fit_transform(list_Review)

reverse_dic = {}
for key in cntizer.vocabulary_:
    reverse_dic[cntizer.vocabulary_[key]] = key
top_30 = np.asarray(np.argsort(np.sum(X_cnt, axis=0))[0,-30:][0, ::-1]).flatten()
top_30words = [reverse_dic[v] for v in top_30]

X_train, X_test, y_train, y_test = train_test_split(list_Review, list_Sentiment, test_size=0.33, random_state=42)

In [11]:
clf=Pipeline([
        ('tfidf_vectorizer', TfidfVectorizer(ngram_range= (1,2),max_df=0.3,min_df=7)),
        ('rf_classifier', RandomForestClassifier(n_estimators=500,verbose=1,n_jobs=-1))
    ])
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

confusion_matrix(y_test,y_pred).tofile('cfmatrix.txt',sep=',')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   18.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.9s finished


In [12]:
print(confusion_matrix(y_test,y_pred))

[[1247  531]
 [ 495 1246]]
